In [1]:
import pandas as pd
import io, collections

from Bio import SeqIO, Entrez, SearchIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML, NCBIWWW

Entrez.email = "A.N.Other@example.com"

In [2]:
df = pd.read_csv('pgen_Plants.csv').fillna('')
df = df[df['Clade'].str.contains('Chlorophyte')]
df['accession'] = ['']*df.shape[0]
df.index = df['Sequence name in figure']
df.head(3)

,Sequence name in figure,Full sequence name,Species,Clade,Origin of sequence,Protein sequence,Protein alignment,CDS sequence,CDS alignment,accession
Sequence name in figure,,,,,,,,,,
Cre12g5045,Cre12g5045,Cre12.g504550 Org_Creinhardtii peptide: Cre12....,Chlamydomonas reinhardtii,Chlorophyte,Phytozome (https://phytozome.jgi.doe.gov/pz/po...,MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKEPSKKAAKEP...,----------------------------------------------...,ATGGCCCCCAAGAAGGACGAGAAGCCCGCGACTCAGGAGGCTGCCG...,,
Cre12g5047,Cre12g5047,Cre12.g504700 Org_Creinhardtii peptide: Cre12....,Chlamydomonas reinhardtii,Chlorophyte,Phytozome (https://phytozome.jgi.doe.gov/pz/po...,MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKE...,----------------------------------------------...,ATGGCCCCCAAGAAGGACGAGAAGCCCGCGACTCAGGAGGCTGCCG...,,
Cre12g5056,Cre12g5056,Cre12.g505600 Org_Creinhardtii peptide: Cre12....,Chlamydomonas reinhardtii,Chlorophyte,Phytozome (https://phytozome.jgi.doe.gov/pz/po...,MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKE...,----------------------------------------------...,ATGGCCCCCAAGAAGGACGAGAAGCCCGCGACTCAGGAGGCTGCCG...,,


In [3]:
sequences = "\n".join([SeqRecord(Seq(row['Protein sequence']), id=row['Sequence name in figure'], name=row['Sequence name in figure'],
                                 description=f"organism={row['Species']}").format("fasta") for i, row in df.iterrows()])
print(sequences)

>Cre12g5045 organism=Chlamydomonas reinhardtii
MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKEPSKKAAKEPKGDGEKKDKKKKKS
AVETYKLYIYKVLKQVHPDTGISSKAMSIMNSFINDIFEKVATEASKLSRYNKKPTVTSR
EIQTAVRLVLPGELAKHAVSEGTKAVTKFTSG

>Cre12g5047 organism=Chlamydomonas reinhardtii
MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKEPKGDGEKKDKKKKK
SAVETYKLYIYKVLKQVHPDTGISSKAMSIMNSFINDIFEKVATEASKLSRYNKKPTVTS
REIQTAVRLVLPGELAKHAVSEGTKAVTKFTSG

>Cre12g5056 organism=Chlamydomonas reinhardtii
MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKEPKGDGEKKDKKKKK
SAVETYKLYIYKVLKQVHPDTGISSKAMSIMNSFINDIFEKVATEASKLSRYNKKPTVTS
REIQTAVRLVLPGELAKHAVSEGTKAVTKFTSG

>Cre12g5062 organism=Chlamydomonas reinhardtii
MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKEPKGDGEKKDKKKKK
SAVETYKLYIYKVLKQVHPDTGISSKAMSIMNSFINDIFEKVATEASKLSRYNKKPTVTS
REIQTAVRLVLPGELAKHAVSEGTKAVTKFTSG

>Cre13g5700 organism=Chlamydomonas reinhardtii
MAPKKDEKPATRAATQEAGAEATAKAEAKPKAEKAAKKAKKEPAKKAAKEPKGDGEKKDK
KKKSAVETYKLYIYKVLKQVHPDTGISSKAMSIMNSFINDIFEKVATEASKLSRYNKKPT
VTSREIQTAVRLVLPG

## ... Blast sequences

In [4]:
df_blast = pd.read_csv(f"9V1SBSGY016-Alignment-HitTable.csv", header=None)
df_blast.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Cre12g5045,XP_001702223.2,100.000,152,0,0,1,152,1,152,1.870000e-100,295,100.00
1,Cre12g5045,XP_001691007.1,99.346,153,0,1,1,152,1,153,1.460000e-98,290,99.35
2,Cre12g5045,XP_001700461.1,98.693,153,1,1,1,152,1,153,5.810000e-98,289,98.69
3,Cre12g5045,XP_042914553.1,98.039,153,2,1,1,152,1,153,6.210000e-97,286,98.04
4,Cre12g5045,XP_001700403.1,98.039,153,2,1,1,152,1,153,6.420000e-97,286,98.04


In [5]:
df_blast = pd.read_csv(f"9V1SBSGY016-Alignment-HitTable.csv", header=None)
df_blast = df_blast.iloc[:,:8]
df_blast = df_blast[(df_blast[2]==100) & (df_blast[6]==1) & (df_blast[7]==df_blast[3])]
for i, row in df_blast.iterrows():
    if len(df.loc[row[0], 'Protein sequence']) != row[3]: continue
    df.at[row[0], 'accession'] = row[1]
df_blast

,0,1,2,3,4,5,6,7
0,Cre12g5045,XP_001702223.2,100.0,152,0,0,1,152
100,Cre12g5047,XP_001691007.1,100.0,153,0,0,1,153
200,Cre12g5056,XP_001691007.1,100.0,153,0,0,1,153
300,Cre12g5062,XP_001691007.1,100.0,153,0,0,1,153
400,Cre13g5700,XP_001693722.1,100.0,156,0,0,1,156
500,Cre13g5907,XP_001693071.1,100.0,155,0,0,1,155
600,Cre13g5912,XP_001692948.1,100.0,155,0,0,1,155
700,Cre17g7086,XP_001691693.1,100.0,153,0,0,1,153
800,Cre17g7091,XP_001691541.1,100.0,153,0,0,1,153
900,Cre17g7104,XP_001696283.1,100.0,153,0,0,1,153


In [6]:
df_blast.shape, len(set(df_blast[0]))

((54, 8), 51)

In [7]:
df_blast[df_blast[0].duplicated()]

,0,1,2,3,4,5,6,7
4701,Vca27s0139,XP_002959847.1,100.0,142,0,0,1,142
4801,Vca27s0143,XP_002959847.1,100.0,142,0,0,1,142
4901,Vca27s0148,XP_002959847.1,100.0,142,0,0,1,142


In [8]:
df_blast[df_blast[0].isin(df_blast[df_blast[0].duplicated()][0])]

,0,1,2,3,4,5,6,7
4700,Vca27s0139,XP_002956800.1,100.0,155,0,0,1,155
4701,Vca27s0139,XP_002959847.1,100.0,142,0,0,1,142
4800,Vca27s0143,XP_002956800.1,100.0,155,0,0,1,155
4801,Vca27s0143,XP_002959847.1,100.0,142,0,0,1,142
4900,Vca27s0148,XP_002956800.1,100.0,155,0,0,1,155
4901,Vca27s0148,XP_002959847.1,100.0,142,0,0,1,142


In [9]:
len(df.loc['Vca27s0139', 'Protein sequence'])

155

In [10]:
df.loc[df_blast[df_blast[0].duplicated()][0]]['accession']

Sequence name in figure
Vca27s0139    XP_002956800.1
Vca27s0143    XP_002956800.1
Vca27s0148    XP_002956800.1
Name: accession, dtype: object

In [11]:
for a in df['accession'].unique():
    if df[df['accession']==a]['Protein sequence'].unique().shape[0] == 1: continue
    print(a)

In [12]:
df.columns

Index(['Sequence name in figure', 'Full sequence name', 'Species', 'Clade',
       'Origin of sequence', 'Protein sequence', 'Protein alignment',
       'CDS sequence', 'CDS alignment', 'accession'],
      dtype='object')

In [13]:
df['Protein sequence'].unique().shape[0]

35

In [14]:
new_df = df[['accession', 'Species', 'Protein sequence']].drop_duplicates()
new_df.shape

(35, 3)

In [15]:
new_df

,accession,Species,Protein sequence
Sequence name in figure,,,
Cre12g5045,XP_001702223.2,Chlamydomonas reinhardtii,MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKEPSKKAAKEP...
Cre12g5047,XP_001691007.1,Chlamydomonas reinhardtii,MAPKKDEKPATQEAAAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKE...
Cre13g5700,XP_001693722.1,Chlamydomonas reinhardtii,MAPKKDEKPATRAATQEAGAEATAKAEAKPKAEKAAKKAKKEPAKK...
Cre13g5907,XP_001693071.1,Chlamydomonas reinhardtii,MAPKKDEKKDAAAPEAAEPKAEKESKPKADKAAKKAKKAPAKKAAK...
Cre13g5912,XP_001692948.1,Chlamydomonas reinhardtii,MAPKKDEKKDAAAPEAAEPKAEKESKPKADKAAKKAKKSPAKKAAK...
Cre17g7086,XP_001691693.1,Chlamydomonas reinhardtii,MAPKKDEKSATQEAGAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKE...
Cre17g7091,XP_001691541.1,Chlamydomonas reinhardtii,MAPKKDEKPATAEAGAEAPAKAEAKPKAEKAGKKAKKEPAKKAAKE...
Cre17g7104,XP_001696283.1,Chlamydomonas reinhardtii,MAPKKDEKPATAEAGAEAPAKADAKPKAEKAAKKAKKEPSKKAAKE...
Cre17g7117,XP_042914553.1,Chlamydomonas reinhardtii,MAPKKDEKPPTQEAGAEAPAKAEAKPKAEKAAKKAKKEPSKKAAKE...


In [17]:
new_df.to_csv('h2b_chlorophyta_histones.csv', index=False)